# gtfParsingForDecapping.ipynb
## Marcus Viscardi,    April 15, 2024

Working in `decappingQuantification.ipynb`, I realized that we need to be able to calculate distances in transcriptome space from the genomic coordinates of our 5TERA libraries.

To do this, we'll want to be able to "walk" along the GTF file as we move through the genomic space, then subtract out the intronic regions to get the distance along the transcript.

A big problem with this is gonna be the fact that most genes have multiple transcripts...

For our NMD guyz, we have previously binned them into target, non-target, and ambiguous. With this, we can figure out which transcripts in the GTF are the NMD targets and the non-targets then work from there? Feels crazy... And doesn't handle the multiple non-target transcripts...

***

Here, we want to see if we can parse the GTF file into something that will be useful for this purpose!

In [29]:
import nanoporePipelineCommon as npCommon

import numpy as np
import pandas as pd

import seaborn as sea
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from tqdm.auto import tqdm

from pprint import pprint

from pathlib import Path

from icecream import ic
from datetime import datetime

pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)

def __time_formatter__():
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"ic: {now} | > "
ic.configureOutput(prefix=__time_formatter__)

_ = ic("Imports done!")

ic: 2024-04-15 11:30:16 | > 'Imports done!'


In [4]:
gtf_parquet_path = Path("/data16/marcus/genomes/plus_cerENO2_elegansRelease100/230327_allChrs_plus-cerENO2.gtf.parquet")

gtf_df = pd.read_parquet(gtf_parquet_path)
gtf_df.query("gene_name == 'ubl-1'")

,chr,source,feature,start,end,score,strand,frame,gene_name,gene_id,gene_version,gene_source,gene_biotype,transcript_id,transcript_source,transcript_biotype,exon_number,exon_id
654649,III,WormBase,gene,3068306,3069210,.,+,.,ubl-1,WBGene00006725,1,WormBase,protein_coding,None,None,None,NaN,None
654650,III,WormBase,transcript,3068306,3069209,.,+,.,ubl-1,WBGene00006725,1,WormBase,protein_coding,H06I04.4a.3,WormBase,protein_coding,NaN,None
654651,III,WormBase,exon,3068306,3068467,.,+,.,ubl-1,WBGene00006725,1,WormBase,protein_coding,H06I04.4a.3,WormBase,protein_coding,1.0,H06I04.4a.3.e1
654652,III,WormBase,CDS,3068335,3068467,.,+,0,ubl-1,WBGene00006725,1,WormBase,protein_coding,H06I04.4a.3,WormBase,protein_coding,1.0,None
654653,III,WormBase,start_codon,3068335,3068337,.,+,0,ubl-1,WBGene00006725,1,WormBase,protein_coding,H06I04.4a.3,WormBase,protein_coding,1.0,None
654654,III,WormBase,exon,3068574,3068743,.,+,.,ubl-1,WBGene00006725,1,WormBase,protein_coding,H06I04.4a.3,WormBase,protein_coding,2.0,H06I04.4a.1.e2
654655,III,WormBase,CDS,3068574,3068743,.,+,2,ubl-1,WBGene00006725,1,WormBase,protein_coding,H06I04.4a.3,WormBase,protein_coding,2.0,None
654656,III,WormBase,exon,3068973,3069209,.,+,.,ubl-1,WBGene00006725,1,WormBase,protein_coding,H06I04.4a.3,WormBase,protein_coding,3.0,H06I04.4a.2.e3
654657,III,WormBase,CDS,3068973,3069158,.,+,0,ubl-1,WBGene00006725,1,WormBase,protein_coding,H06I04.4a.3,WormBase,protein_coding,3.0,None
654658,III,WormBase,stop_codon,3069159,3069161,.,+,0,ubl-1,WBGene00006725,1,WormBase,protein_coding,H06I04.4a.3,WormBase,protein_coding,3.0,None


In [48]:
bed_path = Path("/data16/marcus/genomes/plus_cerENO2_elegansRelease100/230327_allChrs_plus-cerENO2.bed")

bed_dict = {}
with open(bed_path, 'r') as bed_file:
    lines = bed_file.readlines()
    for i, line in tqdm(enumerate(lines), total=len(lines)):
        line = line.strip().split("\t")
        line_dict = {
            "chr": line[0],
            "start": int(line[1]),
            "end": int(line[2]),
            "gene_id": line[3].strip('"'),
            "score": line[4],
            "strand": line[5],
            "source": line[6],
            "feature": line[7],
            "item_rgb": line[8],
        }
        # print(line[9].strip(";").split(";"))
        for item in line[9].strip(";").split("; "):
            try:
                if "=" in item:
                    key, value = item.split("=", 1)
                else:
                    key, value = item.split(" ", 1)
            except ValueError:
                pprint(line)
                print(f"Error on line {i}: {item}")
                raise ValueError(f"Error on line {i}: {item}")
            line_dict[key] = value.strip('"')
        # pprint(line)
        # pprint(line_dict)
        # print("\n\n")
        bed_dict[i] = line_dict
bed_df = pd.DataFrame.from_dict(bed_dict, orient="index")

  0%|          | 0/734026 [00:00<?, ?it/s]

In [68]:
target_gene = 'ubl-1'
gene_df = bed_df.query(f"gene_name == @target_gene")

for trans_id in gene_df.transcript_id.unique():
    print('\n', trans_id)
    print(gene_df.query(f"transcript_id == @trans_id and feature == 'exon'").set_index('exon_id')[['start', 'end', 'feature']])
print(gene_df.query(f"feature == 'exon'").set_index('exon_id')[['start', 'end', 'feature']])


 H06I04.4a.3
                  start      end feature
exon_id                                 
H06I04.4a.3.e1  3068305  3068467    exon
H06I04.4a.1.e2  3068573  3068743    exon
H06I04.4a.2.e3  3068972  3069209    exon

 
Empty DataFrame
Columns: [start, end, feature]
Index: []

 H06I04.4b.1
                  start      end feature
exon_id                                 
H06I04.4b.1.e1  3068309  3068467    exon
H06I04.4b.1.e2  3068581  3068743    exon
H06I04.4b.1.e3  3068972  3069200    exon

 H06I04.4a.1
                  start      end feature
exon_id                                 
H06I04.4a.1.e1  3068316  3068467    exon
H06I04.4a.1.e2  3068573  3068743    exon
H06I04.4a.1.e3  3068972  3069210    exon

 H06I04.4a.2
                  start      end feature
exon_id                                 
H06I04.4a.1.e1  3068316  3068467    exon
H06I04.4a.1.e2  3068573  3068743    exon
H06I04.4a.2.e3  3068972  3069209    exon
                  start      end feature
exon_id                